# Job planet 채용 공고 scrapping 스크립트

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup

import pandas as pd

import time
from tqdm import tqdm_notebook

In [9]:
baseUrl = 'https://www.jobplanet.co.kr'
scroll_num = 3

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

In [10]:
# Function for scrolling

def scroll(driver, scroll_limit):
    '''
    Each scroll gets 9 additional references and it takes 1 second of timeout.
    '''
    n = 1
    prev_height = driver.execute_script('return document.body.scrollHeight')
    while n <= scroll_limit:
        print('scrolling', n, '/', scroll_limit)
        n += 1
        driver.execute_script('window.scrollBy(0, 1000)')
        time.sleep(1)
        curr_height = driver.execute_script('return document.body.scrollHeight')
        if curr_height == prev_height:
            break
        prev_height = curr_height
    time.sleep(5)

In [11]:
# Making a list of target hrefs (job_descriptions)

with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
  driver.implicitly_wait(5)
  driver.get(baseUrl + '/job')
  scroll(driver, scroll_num) # get (1 + 3) * 9 = 36 job descriptions

  print('collecting', (1 + scroll_num) * 9, 'of hrefs')
  job_descriptions = [a.get_attribute('href') for a in driver.find_elements(By.TAG_NAME, 'a') if a.get_attribute('href') and baseUrl + '/job/search' in a.get_attribute('href')]

print(len(job_descriptions), job_descriptions)

scrolling 1 / 3
scrolling 2 / 3
scrolling 3 / 3
collecting 36 of hrefs
36 ['https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250510', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250509', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250268', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250507', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250499', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250486', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250433', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250237', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250373', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250542', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250237', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250373', 'https://www.jobplanet.co.kr/job/search?posting_ids%5B%5D=1250380', 'https://www.jobplanet.co.kr/job/search?p

In [12]:
# Scrapping recruitment data from all hrefs collected

extracted_data = {
    'href': [],
    'title': [],
    'company_name': [],
    'job_location': [],
    'job_classification': [],
    'due_date': [],
    'work_experience': [],
    'skills': [],
    'company_intro': [],
    'main_task': [],
    'qualification': [],
    'preference': [],
    'detail_description': [],
}


with webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) as driver:
    driver.implicitly_wait(5)
    for i in tqdm_notebook(range(len(job_descriptions))):
      driver.get(job_descriptions[i])
      WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'job_apply_section')))

      soup = BeautifulSoup(driver.page_source, 'html.parser')
      apply = soup.find('div', 'job_apply_section')
      detail = soup.find('section', 'recruitment-detail')
      summary = detail.find('dl', 'recruitment-summary__dl')

      extracted_data['href'].append(job_descriptions[i])
      extracted_data['title'].append(apply.find('h1', 'ttl').text)
      extracted_data['company_name'].append(apply.find('span', 'company_name').text)
      extracted_data['job_location'].append(apply.find('span', 'job_location').text)
      extracted_data['job_classification'].append(summary.find(text='직무').find_parent('dt').find_next_sibling().text)
      extracted_data['due_date'].append(summary.find(text='마감일').find_parent('dt').findNextSibling().text)
      extracted_data['work_experience'].append(summary.find(text='경력').find_parent('dt').findNextSibling().text)
      extracted_data['skills'].append(summary.find(text='스킬').find_parent('dt').findNextSibling().text)
      extracted_data['company_intro'].append(detail.find('h3', text='기업 소개').find_next_sibling().text)
      extracted_data['main_task'].append(detail.find('h3', text='주요 업무').find_next_sibling().text)
      extracted_data['qualification'].append(detail.find('h3', text='자격 요건').find_next_sibling().text)
      preference = detail.find('h3', text='우대사항')
      if preference:
          extracted_data['preference'].append(preference.find_next_sibling().text)
      else:
          extracted_data['preference'].append('')
      extracted_data['detail_description'].append(detail)


/var/folders/92/qqyym7k148n9fszn9s5dlt0m0000gn/T/ipykernel_94987/1283732989.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(job_descriptions))):


  0%|          | 0/36 [00:00<?, ?it/s]

In [6]:
extracted_df = pd.DataFrame(extracted_data)
extracted_df.head()

,href,title,company_name,job_location,job_classification,due_date,work_experience,skills,company_intro,main_task,qualification,preference,detail_description
0,https://www.jobplanet.co.kr/job/search?posting...,IoT 조명 및 디바이스 기술영업,아카라코리아(주),서울,기술영업,2023.06.26 D-5,7년 이상,"영업기획, 영업, 영업관리, 기술영업, b2c, B2B",[회사소개] \n아카라(Aqara)는 2009년에 설립하여 자체 브랜드를 통해 스마...,"조명 및 IoT 디바이스 영업 (B2B, B2C)\n• 조명 판매, 유통, 마케팅 ...","• 조명 업계 영업/마케팅 경력 7년 이상\n• B2B, B2C 영업 유경험자\n•...",• 스마트홈 IoT 관련 분야 경험자\n• 전산/컴퓨터 등 IT 계열 학과 전공하신...,"[[[요약], [<dt class=""recruitment-summary__dt""><..."
1,https://www.jobplanet.co.kr/job/search?posting...,IoT 디바이스/솔루션 상품기획 및 판매전략 수립,아카라코리아(주),서울,상품개발/기획/MD,2023.06.26 D-5 (채용시 마감),4년 이상,"영업기획, IT, MD, 상품기획, 판매전략",[회사소개] \n아카라(Aqara)는 2009년에 설립하여 자체 브랜드를 통해 스마...,"IoT 디바이스/솔루션 상품기획 및 판매전략 수립\n• 스마트홈 IoT 시장 분석,...",• IT 신규 상품 기획 및 판매전략 수립 경력 4년 이상 \n• IT 기술 및 산...,• 공학 전공자\n• 영어 또는 중국어 커뮤니케이션(회화) 가능자\n• 서비스/사업...,"[[[요약], [<dt class=""recruitment-summary__dt""><..."
2,https://www.jobplanet.co.kr/job/search?posting...,CX 매니저 / 1~3년,(주)큐엠아이티,서울,고객지원/CS,2023.06.30 D-9 (채용시 마감),1년 이상,"QA, CS, cx",기술로 스포츠 문화를 바꾸는 '큐엠아이티' 입니다. \n\n[Our Story]\n...,[CX파트 소개]\n1. 고객 중심적 사고를 통한 Voice를 적극 제시하는 문화\...,[이런 분과 PLCO를 만들고 싶어요.]\n • CX 매니저로서 1~3년의 경력...,[이런 분이라면 더욱 환영해요!]\n • 채널톡을 사용한 경험이 있으면 좋습니다...,"[[[요약], [<dt class=""recruitment-summary__dt""><..."
3,https://www.jobplanet.co.kr/job/search?posting...,클라우드 서비스 개발자,(주)클루닉스,서울,클라우드 개발,2023.07.20 D-29 (채용시 마감),4년 이상,"python, aws, Linux, Cloud, shell script, 클라우드,...","""Create Innovation Together, Dream Tomorrow To...","Public Cloud 기반 HPC클라우드 서비스 개발 (PaaS, SaaS)\n\...",[기본]\n대졸(4년) 이상\n\n[우대]\n- 관련 분야 전공자\n- 리눅스 및 ...,,"[[[요약], [<dt class=""recruitment-summary__dt""><..."
4,https://www.jobplanet.co.kr/job/search?posting...,프론트엔드 플랫폼 개발자,(주)클루닉스,서울,백엔드 개발,2023.07.20 D-29 (채용시 마감),3년 이상,"jsp, Javascript, HTML, CSS, Linux, jquery, Php...","""Create Innovation Together, Dream Tomorrow To...",ㆍ클라우드 솔루션 개발\nㆍR&D 및 DeepLearning 플랫폼 개발\nㆍ반응형...,[기본]\n-대졸 (4년) 이상\n\n[우대]\n-해당 직무 경험자 우대\n-관련 ...,,"[[[요약], [<dt class=""recruitment-summary__dt""><..."


In [7]:
extracted_df.to_csv('./extracted.csv', index=False)